In [1]:
import pandas as pd
import json
import os

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 1600)

In [2]:
with open('jo1.json','r') as openfile:
    jo1 = json.load(openfile)
    
with open('jo2.json','r') as openfile:
    jo2 = json.load(openfile)
    
with open('jo3.json','r') as openfile:
    jo3 = json.load(openfile)
    
with open('mo1.json','r') as openfile:
    mo1 = json.load(openfile)
    
with open('mo2.json','r') as openfile:
    mo2 = json.load(openfile)
    
with open('mo3.json','r') as openfile:
    mo3 = json.load(openfile)

In [3]:
df_jo1 = pd.json_normalize(jo1, record_path=['results'])
df_jo2 = pd.json_normalize(jo2, record_path=['results'])
df_jo3 = pd.json_normalize(jo3, record_path=['results'])

df_mo1 = pd.json_normalize(mo1, record_path=['results'])
df_mo2 = pd.json_normalize(mo2, record_path=['results'])
df_mo3 = pd.json_normalize(mo3, record_path=['results'])

In [12]:
mo1

{'html_attributions': [],
 'next_page_token': 'AcYSjRiKVD1WCpwylU8gB-rwtqWV-qECMAmhkB9Aa7cSeQ4CylkqGkXZlJomFGI8PWTLR8EthWumrkmvoy89ukv7FKCYgrQTS6IkT9_gqkx0oDYanMC92q75vzyTJc0ee4axW5wrdHPvrP3uPqa5gD4epn3iUwg_AADTdO_hwGfhRia5OCyXE7CuUzgKw0gXPdiYsXHuNrYtoVaXvfMA_hpFSJYgPdSN5wJhBs3npbS9jdxGvw3vyYeAG-wZVJmL-MNWLFmSnpM-IvQ6Eedjb5J-2tPaDz_Mn7aFdr-p16Lej01NovSd1E8K0sOFvfTDO_w_Y3AYdmrhS4sgP-4yaT0Dasga0u6b5KEgJGN2QmAyhEntMfxGqVIldXhUey75gq9hk2XieZpWVwPp9klzEvtdBdyeDA',
 'results': [{'business_status': 'OPERATIONAL',
   'formatted_address': '2112 N Ashland Ave, Chicago, IL 60614, United States',
   'geometry': {'location': {'lat': 41.9209232, 'lng': -87.6690914},
    'viewport': {'northeast': {'lat': 41.92225667989273,
      'lng': -87.66769962010729},
     'southwest': {'lat': 41.91955702010728, 'lng': -87.67039927989272}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/shopping-71.png',
   'icon_background_color': '#4B96F3',
   'icon_mask_base_uri': 'https://maps.gstat

In [4]:
jo_frames = [df_jo1,df_jo2,df_jo3]
df_jo = pd.concat(jo_frames,axis=0)

In [5]:
mo_frames = [df_mo1,df_mo2,df_mo3]
df_mo = pd.concat(mo_frames,axis=0)

In [6]:
df_jo.columns

Index(['business_status', 'formatted_address', 'icon', 'icon_background_color',
       'icon_mask_base_uri', 'name', 'photos', 'place_id', 'price_level',
       'rating', 'reference', 'types', 'user_ratings_total',
       'geometry.location.lat', 'geometry.location.lng',
       'geometry.viewport.northeast.lat', 'geometry.viewport.northeast.lng',
       'geometry.viewport.southwest.lat', 'geometry.viewport.southwest.lng',
       'opening_hours.open_now', 'plus_code.compound_code',
       'plus_code.global_code'],
      dtype='object')

In [7]:
import folium
from folium.plugins import MarkerCluster

In [22]:
data = {'color':['lightblue','darkpurple'],
       'store':['Jewel-Osco','Mariano\'s']}
df_color_lg = pd.DataFrame(data)

In [24]:
folium.FeatureGroup?

In [28]:
m = folium.Map(location=df_jo[["geometry.location.lat", "geometry.location.lng"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Jewel-Osco & Mariano\'s')

for i,r in df_jo.iterrows():
    location = (r["geometry.location.lat"], r["geometry.location.lng"])
    info = (r["name"], r["formatted_address"])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Name: {bold_start}{info[0]}{bold_end}{new_line} \
    Address: {bold_start}{info[1]}{bold_end}'
    folium.Marker(location=location,icon=folium.Icon(color='lightblue'),
                      tooltip=text)\
    .add_to(m)
    folium.Circle(location=location,radius=4800).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))

for i,r in df_mo.iterrows():
    location = (r["geometry.location.lat"], r["geometry.location.lng"])
    info = (r["name"], r["formatted_address"], r["rating"], r['price_level'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Name: {bold_start}{info[0]}{bold_end}{new_line} \
    Address: {bold_start}{info[1]}{bold_end}{new_line}Rating: \
    {bold_start}{info[2]}{bold_end}{new_line}Price Level: {bold_start}{info[3]}'
    folium.Marker(location=location,icon=folium.Icon(color='darkpurple'),
                      tooltip=text)\
    .add_to(m)

lgd_txt = '<span style="color: {col};">{txt}</span>'
    
for i, r in df_color_lg.iterrows():
    fg = folium.FeatureGroup(name=lgd_txt.format(txt=r['store'],col=r['color']))
    m.add_child(fg)
    
folium.map.LayerControl('topleft', collapsed=False).add_to(m)
m

In [29]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [30]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [31]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/grocery_store_merger_analysis
